In [16]:
import pandas as pd
df = pd.read_csv('reddit.csv')

In [17]:
def is_controversial(ratio):
    if ratio <=.6:
        return 1
    else:
        return 0
df['controversial'] = df.apply(lambda x: is_controversial(x['upvote_ratio']), axis=1)

In [18]:
df

,Unnamed: 0,title,upvote_ratio,num_comments,score,subreddit,controversial
0,0,Today is my Birthday. I turned 25. All I want ...,0.50,170.0,0.0,AskReddit,1
1,1,BF wants me to masturbate for him. Do guys act...,0.50,2991.0,0.0,AskReddit,1
2,2,I hear my house mate masturbating. Should I wa...,0.55,3271.0,120.0,AskReddit,1
3,3,"Why do Americans refer to their country as ""fr...",0.57,9045.0,181.0,AskReddit,1
4,4,"Black American parents of reddit, why do you n...",0.70,11759.0,461.0,AskReddit,0
...,...,...,...,...,...,...,...
277728,277728,Apple Glasses Will Reportedly Use Sony's 'Cutt...,0.96,420.0,4162.0,apple,0
277729,277729,TIL Apple Music compensates musicians twice wh...,0.89,462.0,4170.0,apple,0
277730,277730,It is truly absurd that Siri on MacOS still ca...,0.95,386.0,4169.0,apple,0
277731,277731,I lost my AirPod Pros while golfing and Find M...,0.95,191.0,4169.0,apple,0


In [26]:
import transformers as tf

In [20]:
pip install attrs==18.2.0

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install datasets==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 KB 21.1 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install huggingface_hub==0.5.1

  Using cached huggingface_hub-0.5.1-py3-none-any.whl (77 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.5.0
    Uninstalling huggingface-hub-0.5.0:
      Successfully uninstalled huggingface-hub-0.5.0
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install transformers==4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 20.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [25]:
import transformers
from datasets import Dataset
from sklearn.model_selection import train_test_split
to_ds = df[['title','controversial']]
to_ds.rename(columns = {'title':'text', 'controversial':'label'}, inplace = True)
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
to_ds = swap_columns(to_ds,'label','text')
train, test = train_test_split(to_ds, test_size = .25)
ds_test = Dataset.from_pandas(test)
ds_train = Dataset.from_pandas(train)
ds_test = ds_test.remove_columns('__index_level_0__')
ds_train = ds_train.remove_columns('__index_level_0__')

ImportError: cannot import name 'DatasetInfo' from 'huggingface_hub.hf_api' (/usr/local/lib/python3.8/dist-packages/huggingface_hub/hf_api.py)

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
from transformers import DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [13]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


In [14]:
tokenized_test = ds_test.map(preprocess_function, batched=True)
tokenized_train = ds_train.map(preprocess_function, batched=True)

  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/209 [00:00<?, ?ba/s]

In [45]:
tf_train_set = tokenized_train.to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_validation_set = tokenized_test.to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

In [36]:
from transformers import AutoTokenizer
from transformers import create_optimizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use i

In [47]:

batch_size = 8
num_epochs = 8
batches_per_epoch = len(tokenized_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-6, num_warmup_steps=0, num_train_steps=total_train_steps)
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=2,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [2]:
history = model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks = [callback])

NameError: name 'model' is not defined

In [40]:
history

In [34]:
!mkdir -p model2
model.save('model2/my_model')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO:tensorflow:Assets written to: model2/my_model/assets


INFO:tensorflow:Assets written to: model2/my_model/assets


In [39]:

model = TFAutoModelForSequenceClassification.from_pretrained("./model2/my_model/", num_labels=2)

OSError: Can't load config for './model2/my_model/'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './model2/my_model/' is the correct path to a directory containing a config.json file

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns 

In [ ]:

sns.lineplot(data=losses)

In [ ]:
tf.squeeze

In [21]:
def get_input(inp):
    return tokenizer(inp,truncation=True, padding=True, return_tensors="tf")['input_ids']
pred = model.predict(get_input('Cats'))['logits']
if pred[0][0]>pred[0][1]:
    print(0)
else:
    print(1)

0


In [29]:
import gradio as gr

def get_input(inp):
    return tokenizer(inp,truncation=True, padding=True, return_tensors="tf")['input_ids']

def is_controversial(post):
    inp = tokenizer(post,truncation=True, padding=True, return_tensors="tf")['input_ids']
    pred = model.predict(inp)['logits']
    if pred[0][0]>pred[0][1]:
        return "Non Controversial"
    else:
        return "Controversial"


interface = gr.Interface(
    fn=is_controversial,
    live=True,
    inputs=gr.inputs.Textbox(lines=1, placeholder=None, default="", label=None, optional=False),
    outputs="label",
)


interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7863/
Running on public URL: https://17778.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f35d051e220>,
 'http://127.0.0.1:7863/',
 'https://17778.gradio.app')